In [3]:
import os
import cv2
from pathlib import Path
import re
import torch
from torchvision import transforms
from PIL import Image
import torch
import numpy as np
import matplotlib.pyplot as plt
import shutil

In [5]:
def delete_temp_folder(folder_path):
    shutil.rmtree(folder_path)

delete_temp_folder("temp_images")

In [2]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

denormalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225], std=[1/0.229, 1/0.224, 1/0.225])


In [38]:
pil_image = Image.open("test_image.jpg")
np_array = np.array(pil_image)
np_array[0][0], np.max(np_array), np.min(np_array), np_array.shape

(array([72, 48, 38], dtype=uint8), 255, 0, (720, 1280, 3))

In [54]:
t_array = torch.tensor(np_array, dtype = torch.float32)
t_array = t_array.permute(2, 0, 1)
normalized_array = normalize(t_array)

def plot_t(x):
    x = x.clone().detach().cpu().permute(1, 2, 0).numpy()
    plt.imshow(x)


def save_t(x, name):
    x = x.clone().detach().cpu().permute(1, 2, 0).numpy()
    print(x.shape)
    i = Image.fromarray(x)
    print("iamge from arra")
    print(i)
    i.save(name)
# plot_t(normalized_array)
denormalized_array = denormalize(normalized_array)
denormalized_array_n = denormalized_array.type(torch.uint8)

save_t(denormalized_array_n, "normalized.jpg")
# normalized_array[0][0], denormalized_array[0][0], t_array[0][0], denormalized_array_n[0][0]

(720, 1280, 3)
iamge from arra
<PIL.Image.Image image mode=RGB size=1280x720 at 0x163A17591D0>


In [51]:
denormalized_array_n[0][0], denormalized_array_n.shape

(tensor([71, 73, 73,  ..., 55, 55, 50], dtype=torch.uint8),
 torch.Size([3, 720, 1280]))

In [56]:

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_path = 'output_video.mp4'
fps = 30.0  # frames per second
target_size = (640, 480)  # Desired resolution for the video


def extract_image_number(image_name):
    pattern = r'(\d+)\.png'
    match = re.search(pattern, image_name)
    if match:
        return match.group(1)
    else:
        return None

def convert_to_video(images_path,
                     output_path,
                     resolution,
                     ext="png",
                     repeat_frames = 1,
                     fps = 10):
        
        print(f"Converting images to video")

        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, resolution)

        # Getting a list of images from the folder.
        image_files = [str(f) for f in Path(images_path).glob('*.png')] 

        # sort the images
        sorted_images = dict()
        for image in image_files:
            idx = extract_image_number(image)
            if idx:
                sorted_images[int(idx)] = image
        
        del image_files
        sorted_images = dict(sorted(sorted_images.items()))
        # print(sorted_images)
        for image in sorted_images.values():
            # print(image)
            image = cv2.imread(image)
            resized_image = cv2.resize(image, resolution)
            for _ in range(repeat_frames):
                 video_writer.write(resized_image)

        video_writer.release()
        cv2.destroyAllWindows()
        print(f'Video saved to {video_path}')
             

In [57]:
convert_to_video("temp_images",
                 'output.mp4',
                 resolution= (640, 480))

Converting images to video
Video saved to output_video.mp4


In [ ]:
def image_converter(im):
    
    # move the image to cpu
    im_copy = im.cpu()
    
    # for plt.imshow() the channel-dimension is the last
    # therefore use transpose to permute axes
    im_copy = denormalize(im_copy.clone().detach()).numpy()
    im_copy = im_copy.transpose(1,2,0)
    
    # clip negative values as plt.imshow() only accepts 
    # floating values in range [0,1] and integers in range [0,255]
    im_copy = im_copy.clip(0, 1) 
    
    return im_copy